# Stock Management System - Optimización de Producción

Este notebook implementa un sistema para gestionar el inventario y optimizar la producción utilizando programación lineal.

In [3]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
from typing import Dict, Any


In [4]:
class StockManagementSystem:
    def __init__(self, data_path: str):
        try:
            self.df = pd.read_csv(data_path, sep=';', encoding='latin1', skiprows=3, header=0)
            self.df.columns = self.df.iloc[0]
            self.df = self.df[1:]
            self.df.columns = self.df.columns.str.strip()
        except Exception as e:
            raise Exception(f"Error loading CSV: {e}")
        self._clean_data()
        self.SAFETY_STOCK_DAYS = 3
        self.MIN_PRODUCTION_HOURS = 2
    def _clean_data(self):
        self.df = self.df.dropna(how='all').reset_index(drop=True)
    def optimize_production(self, available_hours):
        try:
            c = -1 * np.array(self.df['Cj/H'].values)
            result = linprog(c)
            return result
        except Exception as e:
            raise Exception(f"Optimization error: {e}")


In [5]:
def _clean_data(self):
    try:
        # Eliminar filas vacías y resetear índices
        self.df = self.df.dropna(how='all').reset_index(drop=True)
        self.df = self.df[self.df['COD_ART'] != 'Total general']

        # Convertir columnas relevantes a numéricas
        num_columns = ['Cj/H', 'Disponible', 'Calidad', 'Stock Externo', 'M_Vta -15', 'M_Vta -15 AA']
        for col in num_columns:
            self.df[col] = self.df[col].replace({',': '.'}, regex=True)
            self.df[col] = pd.to_numeric(self.df[col], errors='coerce')

        # Rellenar valores nulos con valores predeterminados
        self.df['Cj/H'] = self.df['Cj/H'].fillna(1)
        self.df['Disponible'] = self.df['Disponible'].fillna(0)
        self.df['Calidad'] = self.df['Calidad'].fillna(0)
        self.df['Stock Externo'] = self.df['Stock Externo'].fillna(0)
        self.df['M_Vta -15'] = self.df['M_Vta -15'].fillna(0)
        self.df['M_Vta -15 AA'] = self.df['M_Vta -15 AA'].fillna(1)

        # Calcular stock total
        self.df['STOCK_TOTAL'] = self.df['Disponible'] + self.df['Calidad'] + self.df['Stock Externo']
        
        # Calcular la estimación de demanda
        self._calculate_demand_estimation()

    except KeyError as e:
        raise Exception(f"Error en las columnas del DataFrame: {e}")
    except Exception as e:
        raise Exception(f"Error al limpiar los datos: {e}")


In [6]:
def _calculate_demand_estimation(self):
    try:
        # Demanda promedio basada en las ventas recientes
        self.df['DEMANDA_MEDIA'] = self.df['M_Vta -15']

        # Variación interanual en porcentaje
        self.df['VAR_ANUAL'] = abs(1 - (self.df['M_Vta -15'] / self.df['M_Vta -15 AA'])) * 100

        # Ajustar demanda para productos con alta variación (>20%)
        mask = self.df['VAR_ANUAL'] > 20
        self.df.loc[mask, 'DEMANDA_MEDIA'] *= (1 + self.df.loc[mask, 'VAR_ANUAL'] / 100)

    except KeyError as e:
        raise Exception(f"Error en las columnas necesarias para la estimación de demanda: {e}")
    except Exception as e:
        raise Exception(f"Error al calcular la estimación de demanda: {e}")


In [7]:
def optimize_production(self, available_hours: float, maintenance_hours: float, pending_orders: Dict[str, float] = None):
    try:
        # Ajustar las horas disponibles tras mantenimiento
        net_hours = available_hours - maintenance_hours

        # Extraer datos relevantes
        production_rates = self.df['Cj/H'].values
        current_stock = self.df['STOCK_TOTAL'].values
        demand = self.df['DEMANDA_MEDIA'].values

        # Definir el objetivo: minimizar el déficit de inventario
        c = -1 * (demand - current_stock)

        # Crear restricciones
        A_ub, b_ub = [], []

        # Restricción de horas totales de producción
        A_ub.append(1 / production_rates)
        b_ub.append(net_hours)

        # Restricción de producción mínima
        min_production = self.MIN_PRODUCTION_HOURS * production_rates
        A_ub.extend(-1 * np.eye(len(production_rates)))
        b_ub.extend(-1 * min_production)

        # Restricción adicional: pedidos pendientes (si se proporcionan)
        if pending_orders:
            pending_array = np.zeros(len(production_rates))
            for code, quantity in pending_orders.items():
                idx = self.df[self.df['COD_ART'] == code].index
                if len(idx) > 0:
                    pending_array[idx[0]] = quantity
            A_pending = -1 * np.eye(len(production_rates))
            b_pending = -1 * (pending_array - current_stock)
            A_ub.extend(A_pending)
            b_ub.extend(b_pending)

        # Resolver el problema de optimización
        result = linprog(
            c,
            A_ub=np.array(A_ub),
            b_ub=np.array(b_ub),
            bounds=(0, None),
            method='highs'
        )

        # Crear el plan de producción
        production_plan = self.df.copy()
        production_plan['CAJAS_PRODUCIR'] = result.x
        production_plan['HORAS_PRODUCCION'] = np.divide(
            result.x, production_plan['Cj/H'], out=np.zeros_like(result.x), where=production_plan['Cj/H'] > 0
        )

        return production_plan

    except Exception as e:
        raise Exception(f"Error al optimizar la producción: {e}")


In [8]:
def generate_production_report(self, production_plan):
    try:
        # Crear un resumen del informe
        report = {
            'total_production': production_plan['CAJAS_PRODUCIR'].sum(),
            'total_hours': production_plan['HORAS_PRODUCCION'].sum(),
            'products_to_produce': production_plan[
                production_plan['CAJAS_PRODUCIR'] > 0
            ][['COD_ART', 'NOM_ART', 'CAJAS_PRODUCIR', 'HORAS_PRODUCCION']].to_dict('records')
        }

        # Exportar el plan de producción a CSV
        production_plan.to_csv("plan_produccion.csv", index=False, sep=';', encoding='latin1')
        print("Plan de producción guardado en 'plan_produccion.csv'.")

        return report

    except Exception as e:
        raise Exception(f"Error al generar el informe de producción: {e}")


In [9]:
if __name__ == "__main__":
    # Ruta del archivo CSV
    data_path = "stock_data.csv"

    # Inicializar el sistema
    system = StockManagementSystem(data_path)

    # Parámetros de entrada
    available_hours = 100
    maintenance_hours = 5
    pending_orders = {"244719": 50, "274756": 30}

    # Optimizar producción
    plan = system.optimize_production(available_hours, maintenance_hours, pending_orders)

    # Generar informe
    report = system.generate_production_report(plan)

    # Imprimir el informe
    print(report)


TypeError: StockManagementSystem.optimize_production() takes 2 positional arguments but 4 were given